### Get 16S sequence embedded into a model,mayneed to not use just a general language model honestly tho....

In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch
import random

In [ ]:
# Single list of sentences - Possible tens of thousands of sentences
sentences = [
    "TGCTTCTTGGGTGTTGAGCGTGACC",
    "GTGATCCCTTCGTAGCACCTGGGAA",
    "GAGCTCGAATAGGTTATCAGAGCAG",
    "CGACCGATGTCAAGAGCGCGCGCGT",
    "TCTTACCTACTTGACTCTAGTGCGG",
    "CCGACTGTTAGTCTCAGTTGTACTA",
    "TGCGAACGTGGACCTGGTGCGAAGC",
    "AGCTTGCGTGGATTCGTAGGGCTGG",
    "CGTCCCGTTGCATATCACGAGAGCG",
    "GGTTACACGTCTACTACTACATCCG"]


paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:10]:
    score, i, j = paraphrase
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))

In [23]:
from Bio import SeqIO
import pandas as pd

# Set the path to your FASTA file
fasta_file = "data/ready4train_seqs.fasta"

# Parse the sequences and headers from the FASTA file
records = SeqIO.parse(fasta_file, "fasta")

# Create a DataFrame from the sequences and headers
df = pd.DataFrame([(record.id, str(record.seq)) for record in records],
                  columns=["id", "sequence"])
df = df.set_index('id')
# Print the first 10 rows of the DataFrame
print(df.head(10))

                                                             sequence
id                                                                   
GB_GCA_009371985.2  GAAGAGTTTGATCCTGGCTCAGGATGAACGCTAGCGGCAGGCCTAA...
RS_GCF_902705855.1  GGAGGTGATCCAGCCCCAGGTTCCCCTAGGGCTACCTTGTTACGAC...
RS_GCF_000426325.1  AGGAGGTGATCCAGCCGCACCTTCCGGTACGGCTACCTTGTTACGA...
RS_GCF_004343615.1  TGAGAGTTTGATCCTGGCTCAGAGCGAACGCTGGCGGCATGCTTAA...
RS_GCF_010727475.1  AGGAGGTGATCCAGCCGCACCTTCCGGTACGGCTACCTTGTTACGA...
RS_GCF_006540305.1  GGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTAA...
RS_GCF_002022265.1  GCAACCCTCTGTACCGACCATTGTAGCACGTGTGTAGCCCAGGCCG...
GB_GCA_016192685.1  CCTTTTTGCTCCCCACGCTTTCGTGCAATGAGAGTCAGTAATGCGC...
RS_GCF_900130015.1  GGAGAGTTTGATCCTGGCTCAGAACGAACGCTGGCGGCGTGCTTAA...
GB_GCA_003288255.1  TTGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCTTA...


In [25]:
taxonomy = pd.read_csv('data/train_taxonomy.tsv', sep='\t', index_col = 0)
taxonomy.head(10)

,domain,phylum,class,order,family,genus,species
Seq_ID,,,,,,,
RS_GCF_003697165.2,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Escherichia,Escherichia coli
RS_GCF_001027105.1,Bacteria,Firmicutes,Bacilli,Staphylococcales,Staphylococcaceae,Staphylococcus,Staphylococcus aureus
RS_GCF_000006945.2,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Salmonella,Salmonella enterica
RS_GCF_000742135.1,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Enterobacteriaceae,Klebsiella,Klebsiella pneumoniae
RS_GCF_001457635.1,Bacteria,Firmicutes,Bacilli,Lactobacillales,Streptococcaceae,Streptococcus,Streptococcus pneumoniae
RS_GCF_000195955.2,Bacteria,Actinobacteriota,Actinomycetia,Mycobacteriales,Mycobacteriaceae,Mycobacterium,Mycobacterium tuberculosis
RS_GCF_001457615.1,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Pseudomonas aeruginosa
RS_GCF_009759685.1,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Moraxellaceae,Acinetobacter,Acinetobacter baumannii
RS_GCF_001077535.1,Bacteria,Firmicutes_A,Clostridia,Peptostreptococcales,Peptostreptococcaceae,Clostridioides,Clostridioides difficile


In [26]:
taxonomy['sequence'] = df['sequence']

In [28]:
# Get rows with at least two occurrences of the value in the "genus" column
genus_counts = taxonomy['genus'].value_counts()
genus_multiple_occurrences = genus_counts[genus_counts >= 2].index.tolist()
df_subset = taxonomy[taxonomy['genus'].isin(genus_multiple_occurrences)]

# Split the DataFrame into two subsets, test and ref, with about 50% of each genus value
test = pd.DataFrame()
ref = pd.DataFrame()
for genus in genus_multiple_occurrences:
    genus_subset = df_subset[df_subset['genus'] == genus]
    n = len(genus_subset)
    indices = np.arange(n)
    np.random.shuffle(indices)
    split_idx = int(np.floor(n / 2))
    test_idx = indices[:split_idx]
    ref_idx = indices[split_idx:]
    test = pd.concat([test, genus_subset.iloc[test_idx]])
    ref = pd.concat([ref, genus_subset.iloc[ref_idx]])

# Print the resulting DataFrames
print(test)
print(ref)

                      domain              phylum                class  \
Seq_ID                                                                  
RS_GCF_019049245.1  Bacteria    Actinobacteriota        Actinomycetia   
RS_GCF_000772045.1  Bacteria    Actinobacteriota        Actinomycetia   
RS_GCF_004117935.1  Bacteria    Actinobacteriota        Actinomycetia   
RS_GCF_010550605.1  Bacteria    Actinobacteriota        Actinomycetia   
RS_GCF_002754715.1  Bacteria    Actinobacteriota        Actinomycetia   
...                      ...                 ...                  ...   
RS_GCF_003852705.1  Bacteria        Bacteroidota          Bacteroidia   
RS_GCF_018135625.1  Bacteria      Proteobacteria  Alphaproteobacteria   
RS_GCF_000429325.1  Bacteria  Desulfobacterota_I     Desulfovibrionia   
GB_GCA_017546175.1  Bacteria     Elusimicrobiota        Elusimicrobia   
GB_GCA_002337145.1  Bacteria        Bacteroidota          Bacteroidia   

                                 order            

In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2') # can i use a biological dna model from hugging face? 

In [40]:
len(test.sequence[:100].tolist())

100

In [51]:
# Initialize variables for evaluation metrics
total_accuracy = 0
count = 0

test_list = test.sequence[:100].tolist()
ref_list = ref.sequence[:100].tolist()

# Single list of sentences - Possible tens of thousands of sentences
sequences = test_list + ref_list
paraphrases = util.paraphrase_mining(model, sequences)  

# Group sublists by their second element (index 1)
grouped_dict = {}
for sublist in paraphrases:
    key = sublist[1]
    if key not in grouped_dict:
        grouped_dict[key] = []
    grouped_dict[key].append(sublist)

# Get the two sublists with the greatest value in element 0 for each unique value in element 1
result = []
for key in sorted(grouped_dict.keys()):
    sublists = grouped_dict[key]
    top_two_sublists = sorted(sublists, key=lambda x: x[0], reverse=True)[:2]
    result.extend(top_two_sublists)

# Replace indices with corresponding sentences
for sublist in result:
    sublist[1] = sequences[sublist[1]]
    sublist[2] = sequences[sublist[2]]

for seq in test:
    # Get all sublists where element 1 equals the match string
    closest_match = [sublist[2] for sublist in result if ((sublist[1] == seq))]
    correct_match = [key for key, values in seq_dict.items() if seq in values] # make sequence dictionary
    accuracy = int(any(item in closest_match for item in correct_match))
    
    # Update evaluation metrics
    total_accuracy += accuracy
    count += 1
    
    print(f"Input taxa: {input_verb}, closest match: {closest_match}, correct match: {correct_match}")

# Compute average evaluation metrics
avg_accuracy = total_accuracy / count

print(f"Average Accuracy: {avg_accuracy}")

NameError: name 'word_dict' is not defined

### can also try below with DNA sequence model

In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Import the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")
model = AutoModelForMaskedLM.from_pretrained("InstaDeepAI/nucleotide-transformer-2.5b-multi-species")

# Create a dummy dna sequence and tokenize it
sequences = ['ATTCTG' * 9]
tokens_ids = tokenizer.batch_encode_plus(sequences, return_tensors="pt")["input_ids"]

# Compute the embeddings
attention_mask = tokens_ids != tokenizer.pad_token_id
torch_outs = model(
    tokens_ids,
    attention_mask=attention_mask,
    encoder_attention_mask=attention_mask,
    output_hidden_states=True
)

# Compute sequences embeddings
embeddings = torch_outs['hidden_states'][-1].detach().numpy()
print(f"Embeddings shape: {embeddings.shape}")
print(f"Embeddings per token: {embeddings}")

# Compute mean embeddings per sequence
mean_sequence_embeddings = torch.sum(attention_mask.unsqueeze(-1)*embeddings, axis=-2)/torch.sum(attention_mask, axis=-1)
print(f"Mean sequence embeddings: {mean_sequence_embeddings}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.53s/it]


Embeddings shape: (1, 10, 2560)
Embeddings per token: [[[-0.04825436  0.21411765  0.07392845 ...  0.05151267  0.01025398
    0.21558578]
  [-0.36657548  0.19614793  0.17926945 ...  0.42881718  0.10352972
   -0.07980208]
  [ 0.33798268  0.1696109  -0.03771623 ...  0.7363859  -0.519177
    0.21427338]
  ...
  [ 0.30988532  0.47271636 -0.08401014 ...  0.7210888  -0.69447356
    0.08499449]
  [ 0.19159164  0.52927226 -0.13266128 ...  0.73496157 -0.63323313
    0.02344994]
  [ 0.20589665  0.3151195  -0.19496821 ...  0.6304233  -0.7212018
    0.01103845]]]
Mean sequence embeddings: tensor([[ 0.1267,  0.4275, -0.0254,  ...,  0.5978, -0.4886,  0.0630]])


### need to get data andfind method to compare label to true or false label, or I can try to get database from green genes and then compare the ability of this method to the classification of greengenes using a real data set

In [11]:
print(sequences)

['ATTCTGATTCTGATTCTGATTCTGATTCTGATTCTGATTCTGATTCTGATTCTG']
